In [1]:
versioninfo()

Julia Version 1.6.2
Commit 1b93d53fc4* (2021-07-14 15:36 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin20.4.0)
  CPU: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, skylake)
Environment:
  JULIA_NUM_THREADS = 6


In [2]:
import CSV
using DataFrames
# using StatsBase
include("pred_funcs.jl")
using MLJ
# import MLJBase
import TableView.showtable
using Plots
import FileIO
# using Statistics

WebIO._IJuliaInit()

In [3]:
1+1

2

In [4]:
# XGBoostClassifier = @load XGBoostClassifier verbosity=0  # loads code defining a model type
# LogisticClassifier = @load LogisticClassifier pkg="MLJLinearModels" verbosity=0  # loads code defining a model type
# AdaBoostStumpClassifier = @load AdaBoostStumpClassifier verbosity=0  # loads code defining a model type
# EvoTreeClassifier = @load EvoTreeClassifier verbosity=0
# GradientBoostingClassifier = @load GradientBoostingClassifier verbosity=0
# EvoTreeGaussian = @load EvoTreeGaussian verbosity=0
# SVC = @load SVC verbosity=0
# LGBMClassifier = @load LGBMClassifier verbosity=0

In [5]:
L=CSV.read("lofreq_start.csv",DataFrame,downcast=true,stringtype=String, missingstring=["","NA", "NAN", "NULL"]);
L.real = [ismissing(i) ? false : i==1 ? true : false for i in L.real];
L.training = [true for i in 1:nrow(L)]

M=CSV.read("mutect_start.csv",DataFrame,downcast=true,stringtype=String, missingstring=["","NA", "NAN", "NULL"]);
M.real = [ismissing(i) ? false : i==1 ? true : false for i in M.real];
M.training = [true for i in 1:nrow(M)]

V=CSV.read("vardict_start.csv",DataFrame,downcast=true,stringtype=String, missingstring=["","NA", "NAN", "NULL"]);
V.real = [ismissing(i) ? false : i==1 ? true : false for i in V.real];
V.training = [true for i in 1:nrow(V)]

V2=CSV.read("vardict_BMT.csv",DataFrame,downcast=false,stringtype=String, missingstring=["","NA", "NAN", "NULL"]);
V2.real = [false for i in 1:nrow(V2) ];
V2.sample_key = V2.subject .*" ".* V2.key
V2.FPpass = [true for i in 1:nrow(V2)]
V2.training = [false for i in 1:nrow(V2)];

P=CSV.read("pindel_start.csv",DataFrame,downcast=true,stringtype=String, missingstring=["","NA", "NAN", "NULL"]);
P.real = [ismissing(i) ? false : i==1 ? true : false for i in P.real];
P.training = [true for i in 1:nrow(P)]

M2=CSV.read("mutect_BMT.csv",DataFrame,downcast=false,stringtype=String, missingstring=["","NA", "NAN", "NULL"]);
M2.real = [false for i in 1:nrow(M2) ];
M2.sample_key = M2.subject .*" ".* M2.key
M2.FPpass = [true for i in 1:nrow(M2)];
M2.training = [false for i in 1:nrow(M2)];

M3=CSV.read("mutect_BMT_old.csv",DataFrame,downcast=true,stringtype=String, missingstring=["","NA", "NAN", "NULL"]);
M3.real = [false for i in 1:nrow(M3) ];
M3.sample_key = M3.subject .*" ".* M3.key;
M3.FPpass = [true for i in 1:nrow(M3)];
M3.training = [false for i in 1:nrow(M3)];


In [6]:
temp=CSV.read("bmt_real_1.csv",DataFrame,downcast=true,stringtype=String, missingstring=["","NA", "NAN", "NULL"]);
temp.sample_key = temp.SAMPLE .*" ".* temp.key;
#filter!(x->ismissing(x.artifact)||x.artifact!=1,temp)
#select!(temp,[:sample_key,:artifact])



In [202]:
select!(P,["sample_key","REF","ALT","VARIANT_CLASS","QUAL_Pindel","FILTER_Pindel",
        "gt_AD_alt_Pindel","gt_AD_ref_Pindel","gt_AF_Pindel",
#         "RefFwd_Pindel","RefRev_Pindel","AltFwd_Pindel","AltRev_Pindel",
        "SYMBOL","pon_pvalue_Pindel","total_greater_than_min_alt_count_Pindel",
        "dust_score","dust_score_3","dust_score_5","dust_score_10","pass_homopolymer_filter","total_Pindel","real","FPpass"    
        ]);
# rename!(P,:FPpass=>:FPpass_Pindel);
# rename!(P,:VARIANT_CLASS=>:VARIANT_CLASS_Pindel);
# rename!(P,:REF=>:REF_Pindel;
# rename!(P,:ALT=>:ALT_Pindel);
# rename!(P,:SYMBOL=>:SYMBOL_Pindel);
# rename!(P,:pass_homopolymer_filter=>:pass_homopolymer_filter_Pindel);
for nam in names(P)
    if ! occursin("Pindel",nam) && nam != "sample_key"
        rename!(P,"$(nam)"=>string(nam,"_Pindel"))
    end
end

In [203]:
select!(V,["sample_key","REF","ALT","VARIANT_CLASS","QUAL_Vardict","FILTER_Vardict",
        "PMEAN",
#         "ReadQual",
        "SBF","ODDRATIO","SN","HIAF","ADJAF","SHIFT3","NM","HICNT","HICOV","gt_AD_alt_Vardict","gt_AD_ref_Vardict",
        "gt_AF_Vardict","RefFwd_Vardict","RefRev_Vardict","AltFwd_Vardict","AltRev_Vardict","SYMBOL","pon_pvalue_Vardict",
        "dust_score","dust_score_3","dust_score_5","dust_score_10","pass_homopolymer_filter","total_Vardict","real","FPpass","total_greater_than_min_alt_count_Vardict"    
        ]);
# rename!(V,:FPpass=>:FPpass_Vardict);
# rename!(V,:VARIANT_CLASS=>:VARIANT_CLASS_Vardict);
# rename!(V,:REF=>:REF_Vardict;
# rename!(V,:ALT=>:ALT_Vardict);
# rename!(V,:SYMBOL=>:SYMBOL_Vardict);
# rename!(V,:pass_homopolymer_filter=>:pass_homopolymer_filter_Vardict);
for nam in names(V)
    if ! occursin("Vardict",nam) && nam != "sample_key"
        rename!(V,"$(nam)"=>string(nam,"_Vardict"))
    end
end

In [204]:
rename!(V2,"QUAL...6"=>"QUAL_Vardict");
select!(V2,["sample_key","REF","ALT","VARIANT_CLASS",
        "QUAL_Vardict",
        "FILTER_Vardict",
        "PMEAN",
#         "ReadQual",
        "SBF","ODDRATIO","SN","HIAF","ADJAF","SHIFT3","NM","HICNT","HICOV","gt_AD_alt_Vardict","gt_AD_ref_Vardict",
        "gt_AF_Vardict","RefFwd_Vardict","RefRev_Vardict","AltFwd_Vardict","AltRev_Vardict","SYMBOL","pon_pvalue_Vardict",
        "dust_score","dust_score_3","dust_score_5","dust_score_10","pass_homopolymer_filter","total_Vardict","real","FPpass","total_greater_than_min_alt_count_Vardict"    
        ]);
# rename!(V,:FPpass=>:FPpass_Vardict);
# rename!(V,:VARIANT_CLASS=>:VARIANT_CLASS_Vardict);
# rename!(V,:REF=>:REF_Vardict;
# rename!(V,:ALT=>:ALT_Vardict);
# rename!(V,:SYMBOL=>:SYMBOL_Vardict);
# rename!(V,:pass_homopolymer_filter=>:pass_homopolymer_filter_Vardict);
for nam in names(V2)
    if ! occursin("Vardict",nam) && nam != "sample_key"
        rename!(V2,"$(nam)"=>string(nam,"_Vardict"))
    end
end

In [205]:
select!(L,["sample_key","SYMBOL","REF","ALT","VARIANT_CLASS","QUAL_Lofreq","FILTER_Lofreq",
        "gt_AD_alt_Lofreq","gt_AD_ref_Lofreq","gt_AF_Lofreq","RefFwd_Lofreq","RefRev_Lofreq","AltFwd_Lofreq","AltRev_Lofreq","pon_pvalue_Lofreq",
        "dust_score","dust_score_3","dust_score_5","dust_score_10","pass_homopolymer_filter","total_Lofreq","real","FPpass","total_greater_than_min_alt_count_Lofreq"         
        ]);
# rename!(L,:FPpass=>:FPpass_Lofreq);
# rename!(L,:VARIANT_CLASS=>:VARIANT_CLASS_Lofreq);
# rename!(L,:REF=>:REF_Lofreq;
# rename!(L,:SYMBOL=>:SYMBOL_Lofreq);
# rename!(L,:ALT=>:ALT_Lofreq);
# rename!(L,:pass_homopolymer_filter=>:pass_homopolymer_filter_Lofreq);
for nam in names(L)
    if ! occursin("Lofreq",nam) && nam != "sample_key"
        rename!(L,"$(nam)"=>string(nam,"_Lofreq"))
    end
end

In [206]:
select!(M,["sample_key","SYMBOL","REF","ALT","VARIANT_CLASS","QUAL_Mutect","FILTER_Mutect",
        "GERMQ","MBQ","MFRL","MMQ","MPOS","ROQ","TLOD","gt_AD_alt_Mutect","gt_AD_ref_Mutect",
        "gt_AF_Mutect","RefFwd_Mutect","RefRev_Mutect","AltFwd_Mutect","AltRev_Mutect","pon_pvalue_Mutect","total_greater_than_min_alt_count_Mutect",
        "dust_score","dust_score_3","dust_score_5","dust_score_10","pass_homopolymer_filter","total_Mutect","real","FPpass"    
        ]);
# rename!(M,:FPpass=>:FPpass_Mutect);
# rename!(M,:VARIANT_CLASS=>:VARIANT_CLASS_Mutect);
# rename!(M,:REF=>:REF_Mutect;
# rename!(M,:SYMBOL=>:SYMBOL_Mutect);
# rename!(M,:ALT=>:ALT_Mutect);
# rename!(M,:pass_homopolymer_filter=>:pass_homopolymer_filter_Mutect);
for nam in names(M)
    if ! occursin("Mutect",nam) && nam != "sample_key"
        rename!(M,"$(nam)"=>string(nam,"_Mutect"))
    end
end

In [207]:
select!(M2,["sample_key","SYMBOL","REF","ALT","VARIANT_CLASS","QUAL_Mutect","FILTER_Mutect",
        "GERMQ","MBQ","MFRL","MMQ","MPOS","ROQ","TLOD","gt_AD_alt_Mutect","gt_AD_ref_Mutect",
        "gt_AF_Mutect","RefFwd_Mutect","RefRev_Mutect","AltFwd_Mutect","AltRev_Mutect","pon_pvalue_Mutect","total_greater_than_min_alt_count_Mutect",
        "dust_score","dust_score_3","dust_score_5","dust_score_10","pass_homopolymer_filter","total_Mutect","real","FPpass"    
        ]);
# rename!(M,:FPpass=>:FPpass_Mutect);
# rename!(M,:VARIANT_CLASS=>:VARIANT_CLASS_Mutect);
# rename!(M,:REF=>:REF_Mutect;
# rename!(M,:SYMBOL=>:SYMBOL_Mutect);
# rename!(M,:ALT=>:ALT_Mutect);
# rename!(M,:pass_homopolymer_filter=>:pass_homopolymer_filter_Mutect);
for nam in names(M2)
    if ! occursin("Mutect",nam) && nam != "sample_key"
        rename!(M2,"$(nam)"=>string(nam,"_Mutect"))
    end
end

In [208]:
select!(M3,["sample_key","SYMBOL","REF","ALT","VARIANT_CLASS","QUAL_Mutect","FILTER_Mutect",
        "GERMQ","MBQ","MFRL","MMQ","MPOS","ROQ","TLOD","gt_AD_alt_Mutect","gt_AD_ref_Mutect",
        "gt_AF_Mutect","RefFwd_Mutect","RefRev_Mutect","AltFwd_Mutect","AltRev_Mutect","pon_pvalue_Mutect","total_greater_than_min_alt_count_Mutect",
        "dust_score","dust_score_3","dust_score_5","dust_score_10","pass_homopolymer_filter","total_Mutect","real","FPpass"    
        ]);
# rename!(M,:FPpass=>:FPpass_Mutect);
# rename!(M,:VARIANT_CLASS=>:VARIANT_CLASS_Mutect);
# rename!(M,:REF=>:REF_Mutect;
# rename!(M,:SYMBOL=>:SYMBOL_Mutect);
# rename!(M,:ALT=>:ALT_Mutect);
# rename!(M,:pass_homopolymer_filter=>:pass_homopolymer_filter_Mutect);
for nam in names(M3)
    if ! occursin("Mutect",nam) && nam != "sample_key"
        rename!(M3,"$(nam)"=>string(nam,"_Mutect"))
    end
end

In [209]:
M = vcat(M,M2,M3);

In [210]:
size(V)

(502976, 34)

In [211]:
size(V2)

(14158, 34)

In [212]:
V = vcat(V,V2);

In [213]:
temp = outerjoin(P,V,on=:sample_key)
temp = outerjoin(temp,L,on=:sample_key)
temp = outerjoin(temp,M,on=:sample_key);
temp.real = temp.real_Lofreq
temp.FPpass = temp.FPpass_Lofreq
temp.VARIANT_CLASS = temp.VARIANT_CLASS_Lofreq
temp.REF = temp.REF_Lofreq
temp.SYMBOL = temp.SYMBOL_Lofreq
temp.ALT = temp.ALT_Lofreq
temp.pass_homopolymer_filter = temp.pass_homopolymer_filter_Lofreq

for i in 1:nrow(temp)
    if ismissing(temp[i,"real"])
        if ! ismissing(temp[i,"real_Pindel"])
            temp[i,"real"] = temp[i,"real_Pindel"]
        elseif ! ismissing(temp[i,"real_Mutect"])
            temp[i,"real"] = temp[i,"real_Mutect"]
        elseif ! ismissing(temp[i,"real_Vardict"])
            temp[i,"real"] = temp[i,"real_Vardict"]
        else
            println(i)
            println(temp[i,"sample_key"])
            println(temp[i,"real"])
            println("real")
#             println(asdfasdf)
        end
    end
    
    if ismissing(temp[i,"FPpass"])
        if ! ismissing(temp[i,"FPpass_Pindel"])
            temp[i,"FPpass"] = temp[i,"FPpass_Pindel"]
        elseif ! ismissing(temp[i,"FPpass_Mutect"])
            temp[i,"FPpass"] = temp[i,"FPpass_Mutect"]
        elseif ! ismissing(temp[i,"FPpass_Vardict"])
            temp[i,"FPpass"] = temp[i,"FPpass_Vardict"]
        else
            println(i)
                        println(temp[i,"sample_key"])
            println(temp[i,"real"])
            println("fpp")
#             println(asdfasdf)
        end
    end
    
    if ismissing(temp[i,"VARIANT_CLASS"])
        if ! ismissing(temp[i,"VARIANT_CLASS_Pindel"])
            temp[i,"VARIANT_CLASS"] = temp[i,"VARIANT_CLASS_Pindel"]
        elseif ! ismissing(temp[i,"VARIANT_CLASS_Mutect"])
            temp[i,"VARIANT_CLASS"] = temp[i,"VARIANT_CLASS_Mutect"]
        elseif ! ismissing(temp[i,"VARIANT_CLASS_Vardict"])
            temp[i,"VARIANT_CLASS"] = temp[i,"VARIANT_CLASS_Vardict"]
        else
            println(i)
            println(temp[i,"sample_key"])
            println(temp[i,"real"])
            println("vc")
            temp[i,"VARIANT_CLASS"] = "deletion"
#             println(asdfasdf)
        end
    end
    
    if ismissing(temp[i,"REF"])
        if ! ismissing(temp[i,"REF_Pindel"])
            temp[i,"REF"] = temp[i,"REF_Pindel"]
        elseif ! ismissing(temp[i,"REF_Mutect"])
            temp[i,"REF"] = temp[i,"REF_Mutect"]
        elseif ! ismissing(temp[i,"REF_Vardict"])
            temp[i,"REF"] = temp[i,"REF_Vardict"]
        else
            println(i)
                        println(temp[i,"sample_key"])
            println(temp[i,"real"])
            println("ref")
#             println(asdfasdf)
        end
    end
    
    if ismissing(temp[i,"SYMBOL"])
        if ! ismissing(temp[i,"SYMBOL_Pindel"])
            temp[i,"SYMBOL"] = temp[i,"SYMBOL_Pindel"]
        elseif ! ismissing(temp[i,"SYMBOL_Mutect"])
            temp[i,"SYMBOL"] = temp[i,"SYMBOL_Mutect"]
        elseif ! ismissing(temp[i,"SYMBOL_Vardict"])
            temp[i,"SYMBOL"] = temp[i,"SYMBOL_Vardict"]
        else
            if temp[i,"sample_key"] == "761809_1_5000 2:25464501 GT>G"
                temp[i,"SYMBOL"] = "DNMT3A"
            end
            if temp[i,"sample_key"] == "736399_1_5000 4:106196888 CCACCC>ATCT"
                temp[i,"SYMBOL"] = "TET2"
            end
            println(i)
                        println(temp[i,"sample_key"])
            println(temp[i,"real"])
            println("symbol")
#             println(asdfasdf)
        end
    end
    
    if ismissing(temp[i,"ALT"])
        if ! ismissing(temp[i,"ALT_Pindel"])
            temp[i,"ALT"] = temp[i,"ALT_Pindel"]
        elseif ! ismissing(temp[i,"ALT_Mutect"])
            temp[i,"ALT"] = temp[i,"ALT_Mutect"]
        elseif ! ismissing(temp[i,"ALT_Vardict"])
            temp[i,"ALT"] = temp[i,"ALT_Vardict"]
        else
            println(i)
                        println(temp[i,"sample_key"])
            println(temp[i,"real"])
            println("alt")
#             println(asdfasdf)
        end
    end
    
    if ismissing(temp[i,"pass_homopolymer_filter"])
        if ! ismissing(temp[i,"pass_homopolymer_filter_Pindel"])
            temp[i,"pass_homopolymer_filter"] = temp[i,"pass_homopolymer_filter_Pindel"]
        elseif ! ismissing(temp[i,"pass_homopolymer_filter_Mutect"])
            temp[i,"pass_homopolymer_filter"] = temp[i,"pass_homopolymer_filter_Mutect"]
        elseif ! ismissing(temp[i,"pass_homopolymer_filter_Vardict"])
            temp[i,"pass_homopolymer_filter"] = temp[i,"pass_homopolymer_filter_Vardict"]
        else
            temp[i,"pass_homopolymer_filter"] = false
            println(i)
                        println(temp[i,"sample_key"])
            println(temp[i,"real"])
            println("pmf")
#             println(asdfasdf)
        end
    end
    
            
end

# temp.real = temp.real_Lofreq
# temp.FPpass = temp.FPpass_Lofreq
# temp.VARIANT_CLASS = temp.VARIANT_CLASS_Lofreq
# temp.REF = temp.REF_Lofreq
# temp.SYMBOL = temp.SYMBOL_Lofreq
# temp.ALT = temp.ALT_Lofreq
# temp.pass_homopolymer_filter = temp.pass_homopolymer_filter_Lofreq

select!(temp, 

    Not([
            "real_Lofreq",
            "VARIANT_CLASS_Lofreq",
            "FPpass_Lofreq",
            "REF_Lofreq",
            "SYMBOL_Lofreq",
            "ALT_Lofreq",
            "pass_homopolymer_filter_Lofreq",
            
            "real_Mutect",
            "VARIANT_CLASS_Mutect",
            "FPpass_Mutect",
            "REF_Mutect",
            "SYMBOL_Mutect",
            "ALT_Mutect",
            "pass_homopolymer_filter_Mutect",
            
            "real_Vardict",
            "VARIANT_CLASS_Vardict",
            "FPpass_Vardict",
            "REF_Vardict",
            "SYMBOL_Vardict",
            "ALT_Vardict",
            "pass_homopolymer_filter_Vardict",
            
            "real_Pindel",
            "VARIANT_CLASS_Pindel",
            "FPpass_Pindel",
            "REF_Pindel",
            "SYMBOL_Pindel",
            "ALT_Pindel",
            "pass_homopolymer_filter_Pindel",
            
            "QUAL_Pindel",
            "QUAL_Mutect"
            ])

);

907
736399_1_5000 4:106196888 CCACCC>ATCT
true
vc
907
736399_1_5000 4:106196888 CCACCC>ATCT
true
symbol
907
736399_1_5000 4:106196888 CCACCC>ATCT
true
pmf
908
761809_1_5000 2:25464501 GT>G
true
vc
908
761809_1_5000 2:25464501 GT>G
true
symbol
908
761809_1_5000 2:25464501 GT>G
true
pmf


In [214]:
temp.ref_len = float.(length.(temp.REF))
temp.alt_len = float.(length.(temp.ALT))


temp.fwdbias_Mutect = temp.AltFwd_Mutect ./ temp.RefFwd_Mutect
temp.revbias_Mutect = temp.AltRev_Mutect ./ temp.RefRev_Mutect
# temp.fwdbias_Pindel = temp.AltFwd_Pindel ./ temp.RefFwd_Pindel
# temp.revbias_Pindel = temp.AltRev_Pindel ./ temp.RefRev_Pindel
temp.fwdbias_Lofreq = temp.AltFwd_Lofreq ./ temp.RefFwd_Lofreq
temp.revbias_Lofreq = temp.AltRev_Lofreq ./ temp.RefRev_Lofreq
temp.fwdbias_Vardict = temp.AltFwd_Vardict ./ temp.RefFwd_Vardict
temp.revbias_Vardict = temp.AltRev_Vardict ./ temp.RefRev_Vardict


temp.fwdbias_Mutect = [!ismissing(i) && isnan(i) ? 0.0 : i for i in temp.fwdbias_Mutect]
temp.revbias_Mutect = [!ismissing(i) && isnan(i) ? 0.0 : i for i in temp.revbias_Mutect]
temp.fwdbias_Lofreq = [!ismissing(i) && isnan(i) ? 0.0 : i for i in temp.fwdbias_Lofreq]
temp.revbias_Lofreq = [!ismissing(i) && isnan(i) ? 0.0 : i for i in temp.revbias_Lofreq]
temp.fwdbias_Vardict = [!ismissing(i) && isnan(i) ? 0.0 : i for i in temp.fwdbias_Vardict]
temp.revbias_Vardict = [!ismissing(i) && isnan(i) ? 0.0 : i for i in temp.revbias_Vardict]


temp.REF = [length(i)>1 ? "I" : i for i in temp.REF];
temp.ALT = [length(i)>1 ? "I" : i for i in temp.ALT];
println(nrow(temp))

# filter!(x->ismissing(x.gt_AF_Vardict) || x.gt_AF_Vardict>0.001,temp);
# println(nrow(temp))
# filter!(x->ismissing(x.gt_AF_Lofreq) || x.gt_AF_Lofreq>0.001,temp);
# println(nrow(temp))
# filter!(x->ismissing(x.gt_AF_Mutect) || x.gt_AF_Mutect>0.001,temp);
# println(nrow(temp))
# filter!(x->ismissing(x.gt_AF_Pindel) || x.gt_AF_Pindel>0.001,temp);
# println(nrow(temp))


1505402


In [233]:
v = copy(temp);
v.pass_homopolymer_filter = [ismissing(i) ? missing : i for i in v.pass_homopolymer_filter]
v.FPpass = [ismissing(i) ? missing : i for i in v.FPpass]
v.real = [ismissing(i) ? missing : i for i in v.real]

v.gt_AD_ref_Pindel = float.(v.gt_AD_ref_Pindel)
v.gt_AD_alt_Pindel = float.(v.gt_AD_alt_Pindel)
v.gt_AF_Pindel = float.(v.gt_AF_Pindel)
v.pon_pvalue_Pindel = float.(v.pon_pvalue_Pindel)
v.total_greater_than_min_alt_count_Pindel = float.(v.total_greater_than_min_alt_count_Pindel)
v.total_Pindel = float.(v.total_Pindel)

v.QUAL_Vardict = float.(v.QUAL_Vardict)
v.PMEAN_Vardict = float.(v.PMEAN_Vardict)
# v.ReadQual_Vardict = float.(v.ReadQual_Vardict)
v.SBF_Vardict = float.(v.SBF_Vardict)
v.ODDRATIO_Vardict = float.(v.ODDRATIO_Vardict)
v.SN_Vardict = float.(v.SN_Vardict)
v.HIAF_Vardict = float.(v.HIAF_Vardict)
v.ADJAF_Vardict = float.(v.ADJAF_Vardict)
v.SHIFT3_Vardict = float.(v.SHIFT3_Vardict)
v.NM_Vardict = float.(v.NM_Vardict)
v.HICNT_Vardict = float.(v.HICNT_Vardict)
v.HICOV_Vardict = float.(v.HICOV_Vardict)
v.gt_AD_ref_Vardict = float.(v.gt_AD_ref_Vardict)
v.gt_AD_alt_Vardict = float.(v.gt_AD_alt_Vardict)
v.gt_AF_Vardict = float.(v.gt_AF_Vardict)
v.pon_pvalue_Vardict = float.(v.pon_pvalue_Vardict)
v.RefFwd_Vardict = float.(v.RefFwd_Vardict)
v.RefRev_Vardict = float.(v.RefRev_Vardict)
v.AltFwd_Vardict = float.(v.AltFwd_Vardict)
v.AltRev_Vardict = float.(v.AltRev_Vardict)
v.total_greater_than_min_alt_count_Vardict = float.(v.total_greater_than_min_alt_count_Vardict)
v.total_Vardict = float.(v.total_Vardict)

v.QUAL_Lofreq = float.(v.QUAL_Lofreq)
v.gt_AD_ref_Lofreq = float.(v.gt_AD_ref_Lofreq)
v.gt_AD_alt_Lofreq = float.(v.gt_AD_alt_Lofreq)
v.gt_AF_Lofreq = float.(v.gt_AF_Lofreq)
v.pon_pvalue_Lofreq = float.(v.pon_pvalue_Lofreq)
v.RefFwd_Lofreq = float.(v.RefFwd_Lofreq)
v.RefRev_Lofreq = float.(v.RefRev_Lofreq)
v.AltFwd_Lofreq = float.(v.AltFwd_Lofreq)
v.AltRev_Lofreq = float.(v.AltRev_Lofreq)
v.total_greater_than_min_alt_count_Lofreq = float.(v.total_greater_than_min_alt_count_Lofreq)
v.total_Lofreq = float.(v.total_Lofreq)
        
v.GERMQ_Mutect = float.(v.GERMQ_Mutect)
temp2 = [ismissing(i) ? [missing,missing] : split(i,",") for i in v.MBQ_Mutect]
v.MBQ_Mutect_1 = [ismissing(i[1]) ? missing : parse(Float64,i[1]) for i in temp2]
v.MBQ_Mutect_2 = [ismissing(i[2]) ? missing : parse(Float64,i[2]) for i in temp2]

temp2 = [ismissing(i) ? [missing,missing] : split(i,",") for i in v.MFRL_Mutect]
v.MFRL_Mutect_1 = [ismissing(i[1]) ? missing : parse(Float64,i[1]) for i in temp2]
v.MFRL_Mutect_2 = [ismissing(i[2]) ? missing : parse(Float64,i[2]) for i in temp2]

temp2 = [ismissing(i) ? [missing,missing] : split(i,",") for i in v.MMQ_Mutect]
v.MMQ_Mutect_1 = [ismissing(i[1]) ? missing : parse(Float64,i[1]) for i in temp2]
v.MMQ_Mutect_2 = [ismissing(i[2]) ? missing : parse(Float64,i[2]) for i in temp2]

v.MPOS_Mutect = float.(v.MPOS_Mutect)
v.ROQ_Mutect = float.(v.ROQ_Mutect)
v.TLOD_Mutect = float.(v.TLOD_Mutect)
v.gt_AD_ref_Mutect = float.(v.gt_AD_ref_Mutect)
v.gt_AD_alt_Mutect = float.(v.gt_AD_alt_Mutect)
v.gt_AF_Mutect = float.(v.gt_AF_Mutect)
v.pon_pvalue_Mutect = float.(v.pon_pvalue_Mutect)
v.RefFwd_Mutect = float.(v.RefFwd_Mutect)
v.RefRev_Mutect = float.(v.RefRev_Mutect)
v.AltFwd_Mutect = float.(v.AltFwd_Mutect);
v.AltRev_Mutect = float.(v.AltRev_Mutect);
v.total_greater_than_min_alt_count_Mutect = float.(v.total_greater_than_min_alt_count_Mutect);
v.total_Mutect = float.(v.total_Mutect);

select!(v, Not([:MBQ_Mutect,:MFRL_Mutect,:MMQ_Mutect]));


In [234]:
v.FILTER_Pindel = [ismissing(i) ? "Not_Detected" : i=="Not Detected" ? "Not_Detected" : i for i in v.FILTER_Pindel];
v.FILTER_Lofreq = [ismissing(i) ? "Not_Detected" : i=="Not Detected" ? "Not_Detected" : i for i in v.FILTER_Lofreq];
v.FILTER_Mutect = [ismissing(i) ? "Not_Detected" : i=="Not Detected" ? "Not_Detected" : i for i in v.FILTER_Mutect];
v.FILTER_Vardict = [ismissing(i) ? "Not_Detected" : i=="Not Detected" ? "Not_Detected" : i for i in v.FILTER_Vardict];

In [235]:
# for nam in names(temp)
#     println(nam," ",sum(ismissing.(temp[:,nam]))/nrow(temp) ) 
#     println()
# end

In [236]:
# v = hcat(
#     v,
#     textonehot(v,"FILTER_Lofreq"),
# textonehot(v,"FILTER_Vardict"),
# textonehot(v,"REF"),
#     textonehot(v,"FILTER_Pindel"),
# textonehot(v,"FILTER_Mutect"),
# textonehot(v,"VARIANT_CLASS"),
#     textonehot(v,"SYMBOL"),
# textonehot(v,"ALT")
# )

# select!(v,Not(["FILTER_Lofreq","FILTER_Vardict","FILTER_Pindel","FILTER_Mutect","SYMBOL","ALT","REF",
#             "VARIANT_CLASS"]) );

# v.FPpass = float.(v.FPpass);
# v.pass_homopolymer_filter = float.(v.pass_homopolymer_filter);

In [237]:
# CSV.write("combined.csv",v)

In [238]:
sample_key = v.sample_key;
select!(v, Not(:sample_key) );

In [239]:
sum(v.real)

155

In [248]:
y,x=unpack(coerce(v,:real=>OrderedFactor),
    ==(:real),
    colname -> true, );
# train, test = partition(eachindex(y), 0.7, stratify=y,  shuffle=true, rng=12344);
# x,y,train = sampler(x,y,train,"over",0);

In [249]:
nrow(x)

1505402

In [250]:
# ct=0
# for j in 1:ncol(x),i in 1:nrow(x)
    
#     if ismissing(x[i,j])
#         ct+=1
#         x[i,j] = missing
#     end
 
# end
# println(ct)

In [251]:
# transform_standardizer!(x,fit_standardizer(x[train,:]));
# transform_normalizer!(x,fit_normalizer(x[train,:]))

In [252]:
x = hcat(
    x,
#     textonehot(x,"FILTER_Lofreq"),
# textonehot(x,"FILTER_Vardict"),
textonehot(x,"REF"),
#     textonehot(x,"FILTER_Pindel"),
# textonehot(x,"FILTER_Mutect"),
# textonehot(x,"VARIANT_CLASS"),
#     textonehot(x,"SYMBOL"),
textonehot(x,"ALT")
)

select!(x,Not(["FILTER_Lofreq","FILTER_Vardict","FILTER_Pindel","FILTER_Mutect","SYMBOL","ALT","REF",
            "VARIANT_CLASS"]) );

x.FPpass = float.(x.FPpass);
x.pass_homopolymer_filter = float.(x.pass_homopolymer_filter);

In [253]:
for j in 1:ncol(x), i in 1:nrow(x)

    if !ismissing(x[i,j]) && isnan(x[i,j])
        x[i,j] = missing
    end
end

In [254]:
x

,gt_AD_alt_Pindel,gt_AD_ref_Pindel,gt_AF_Pindel,pon_pvalue_Pindel,total_greater_than_min_alt_count_Pindel
,Float64?,Float64?,Float64?,Float64?,Float64?
1,missing,missing,missing,missing,missing
2,2.0,1621.0,0.00123229,0.119066,missing
3,missing,missing,missing,missing,missing
4,98.0,4771.0,0.0201273,1.6717e-35,6.0
5,1.0,3460.0,0.000288934,0.0439059,missing
6,missing,missing,missing,missing,missing
7,missing,missing,missing,missing,missing
8,missing,missing,missing,missing,missing
9,missing,missing,missing,missing,missing


In [255]:
FileIO.save("cleaned_data_2_no_filter.jld2", Dict("sample_key"=>sample_key,"x" => x,
        "y" => y, 
#         "train" => train, "test" => test
    ) ; compress = true
)
